In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
masterMsGraderResponses = pd.read_excel('S22 Grader Positions (Responses).xlsx', sheet_name=['Grace Response Test', 'Grace - Preference Count'])
preferenceCount = masterMsGraderResponses['Grace - Preference Count']
allResponses = masterMsGraderResponses['Grace Response Test']
allResponses.insert(27, "Grade for this class", 'placeholder')
allResponses.insert(28, "Grade to number", 'placeholder1')
allResponses.insert(29, "Semesters to number", 'placeholder2')

courseList = preferenceCount['Course Name']
allFinalSheets = {}

allColumns = ["Index", "Timestamp", "Email Address", "Name", "Email", "SPIRE ID", "Current 8 hour grader", "Current 4 hour grader", "Current GPA", "Entering Semester", "Courses Preferred - please select up to 4", "Please list your grade in each of the courses selected. If you have not taken the course, please explain your qualifications. Preference will be given to students who have taken the course.", "Grade 1", "Grade 2", "Grade 3", "Grade 4", "How many semesters have you been a grader in CICS (not elsewhere)?", "If you have graded before in CICS, which course(s)?", "Do you have industry experience in any of the following areas?", "What motivates you to do an excellent job as a grader?", "Do you have prior experience teaching or tutoring? If so, please describe.", "Please attach a copy your CV below", "Will you graduate at the end of Spring 2022?", "Are you applying for OPT?", "Will you be on CPT in Winter 21/22?", "If so, what is your end date?", "Are you planning to apply for CPT in Summer 22?", "Other notes or things we should know:", "Course Grade", "Grade to number", "Semesters to number"]

for courseName in courseList:
    allFinalSheets[courseName] = pd.DataFrame(columns=allColumns)
    


In [ ]:
def gradeToNumber(grade):
    if (grade == 'A'):
        return 1
    if (grade == 'A-'):
        return 1
    if (grade == 'B+'):
        return 2
    if (grade == 'No course qualification'):
        return 10
    if (grade == 'current'):
        return 4
    else:
        return 3

In [ ]:
def semestersGradedToNumber(semesters):
    if (semesters == '0 (this would be my first semester)'):
        return 0
    if (semesters == '3 or more'):
        return 3
    else: 
        return semesters

In [ ]:
# Organize all applications and store applications in their own object. 
# Keys: SPIRE ID and 
# value: object representing the student with fields, umassEmail, spireId, fullName, curr8Hour, curr4Hour, GPA <any TBD will be counted as 0>, enteringSemester, coursesPreferred
applicants = {}

for row in allResponses.itertuples(index=True):
    umassEmail = row[2]
    fullName = row[3]
    spireId = row[5]
    curr8Hour = row[6]
    curr4Hour = row[7]
    gpa = row[8]
    enteringSemester = row[9]
    preferences = row[10]
    grades = row[11]
    preferencesArray = preferences.split(', ')
    
    grade1 = row[12]
    grade2 = row[13]
    grade3 = row[14]
    grade4 = row[15]
    renameScheme = {}
    i = 0

    for word in allColumns:
        renameScheme[i] = word
        i += 1

    allGrades = [grade1,grade2,grade3,grade4]
    allGradesFiltered = [grade for grade in allGrades if (type(grade) == str)]
    allGradesMap = map(lambda grade: gradeToNumber, allGradesFiltered)

    if (curr8Hour == False):
        for pref, gr in zip(preferencesArray, allGradesFiltered):
            df = (pd.DataFrame(row)).transpose()
            df.rename(columns=renameScheme, inplace=True)
            df["Course Grade"] = gr
            df["Grade to number"] = gradeToNumber(gr)
            df["Semesters to number"] = semestersGradedToNumber(row[16])
            if (enteringSemester == 'Fall 2021'):
                df["Current GPA"] = 0

            allFinalSheets[pref] = allFinalSheets[pref].append(df, ignore_index=True)



In [ ]:
for sheetName in allFinalSheets:
    courseGrade = allFinalSheets[sheetName].pop('Course Grade')
    allFinalSheets[sheetName].insert(9, 'Course Grade', courseGrade)
    allFinalSheets[sheetName] = allFinalSheets[sheetName].sort_values(by=["Grade to number", "Semesters to number", "Current GPA"], ascending=(True, True, False))
    sheetNameArr = sheetName.split(':')
    allFinalSheets[sheetName].to_excel(f"{sheetNameArr[0]}.xlsx")

